In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Conv2D, Input
from qkeras import QConv2D, QDense, QActivation, quantized_bits

2024-02-21 00:15:11.605894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import tensorflow_datasets as tfds

# Pobieranie datasetu Cats vs Dogs
ds, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)


train_ds, test_ds = ds['train'], ds['test']

train_ds = train_ds
test_ds = test_ds

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [4]:
model = Sequential([
  # Pierwsza warstwa konwolucyjna
  QConv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1),
          kernel_quantizer=quantized_bits(8, 0, 1),
          bias_quantizer=quantized_bits(8, 0, 1)),
  # Druga warstwa konwolucyjna
  QConv2D(64, (3, 3), activation='relu',
          kernel_quantizer=quantized_bits(8, 0, 1),
          bias_quantizer=quantized_bits(8, 0, 1)),
  # Warstwa wypłaszczająca
  Flatten(),
  # Warstwa gęsta z funkcją aktywacji softmax
  QDense(10, activation='softmax',
         kernel_quantizer=quantized_bits(8, 0, 1),
         bias_quantizer=quantized_bits(8, 0, 1))
])

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
import numpy as np
acc = np.zeros((8, 5))
for j in range(0, 5):
    for i in range(0, 8):
        bits = (i+1)
        model = Sequential([
            # Pierwsza warstwa konwolucyjna
            QConv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1),
                    kernel_quantizer=quantized_bits(i, 0, 1),
                    bias_quantizer=quantized_bits(i, 0, 1)),
            # Druga warstwa konwolucyjna
            QConv2D(64, (3, 3), activation='relu',
                    kernel_quantizer=quantized_bits(i, 0, 1),
                    bias_quantizer=quantized_bits(i, 0, 1)),
            # Warstwa wypłaszczająca
            Flatten(),
            # Warstwa gęsta z funkcją aktywacji softmax
            QDense(10, activation='softmax',
                    kernel_quantizer=quantized_bits(i, 0, 1),
                    bias_quantizer=quantized_bits(i, 0, 1))
        ])

        model.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])
        model.fit(train_ds, epochs=1)
        acc[i, j] = model.evaluate(test_ds)[1]
        
])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2127076605.py, line 18)